# 리플렉션

[Reflexion](https://arxiv.org/abs/2303.11366) 논문은 언어 피드백과 스스로의 반성을 통해 학습하도록 설계된 아키텍처입니다. 에이전트는 작업에 대한 자신의 응답을 명시적으로 비판하여 더 높은 품질의 최종 답변을 생성하지만, 그만큼 실행 시간이 길어집니다.

![reflexion diagram](attachment:2f424259-8d89-4f4e-94c4-d668a36d8ca2.png)

논문에서는 세 가지 주요 구성 요소를 설명합니다:

1. 자기 반성을 수행하는 액터(에이전트)
2. 과제별 외부 평가자(예: 코드 컴파일 단계)
3. (1)의 반성을 저장하는 에피소드 메모리

공개된 코드에서는 마지막 두 구성 요소가 과제에 매우 특화되어 있으므로, 이 노트북에서는 LangGraph를 사용해 **액터** 부분을 구현합니다.

그래프 정의 부분으로 바로 가려면 아래의 [그래프 구성](#Construct-Graph) 섹션을 참고하세요.

## 환경 설정

`langgraph`(프레임워크), `langchain_openai`(LLM용), 그리고 `langchain` + `tavily-python`(검색 엔진용)을 설치합니다.

이 예제에서는 tavily 검색을 도구로 사용합니다. [여기](https://app.tavily.com/sign-in)에서 API 키를 받거나 원하는 다른 도구로 교체해도 됩니다.

In [ ]:
%pip install -U --quiet langgraph langchain_anthropic tavily-python

In [ ]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


_set_if_undefined("ANTHROPIC_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하면 LangGraph 프로젝트의 문제를 빠르게 파악하고 성능을 향상시킬 수 있습니다. 추적 데이터를 활용해 디버그, 테스트, 모니터링할 수 있으니 <a href="https://docs.smith.langchain.com">여기</a>에서 시작해 보세요.
    </p>
</div>

### LLM 정의

In [ ]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
# You could also use OpenAI or another provider
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4-turbo-preview")

## 반성 기능을 갖춘 액터

Reflexion의 핵심 요소는 "액터"로, 자신이 생성한 답변을 되돌아보고 자기 비판을 통해 다시 실행하며 성능을 향상시키는 에이전트입니다. 주요 하위 구성 요소는 다음과 같습니다.
1. 도구 실행
2. 초기 응답자: 초기 답변 생성(및 자기 반성)
3. 수정자: 이전 반성을 바탕으로 다시 답변

먼저 도구 실행 컨텍스트를 정의해 보겠습니다.

#### Construct tools

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

#### 초기 응답자

<div class="admonition note">
    <p class="admonition-title">LangChain에서 Pydantic 사용</p>
    <p>
        이 노트북은 Pydantic v2 <code>BaseModel</code>을 사용하므로 <code>langchain-core >= 0.3</code> 버전이 필요합니다. 만약 <code>langchain-core < 0.3</code>을 사용하면 Pydantic v1과 v2 <code>BaseModel</code>이 혼합되어 오류가 발생합니다.
    </p>
</div>

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import ValidationError

from pydantic import BaseModel, Field


class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous")


class AnswerQuestion(BaseModel):
    """Answer the question. Provide an answer, reflection, and then follow up with search queries to improve the answer."""

    answer: str = Field(description="~250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    search_queries: list[str] = Field(
        description="1-3 search queries for researching improvements to address the critique of your current answer."
    )


class ResponderWithRetries:
    def __init__(self, runnable, validator):
        self.runnable = runnable
        self.validator = validator

    def respond(self, state: dict):
        response = []
        for attempt in range(3):
            response = self.runnable.invoke(
                {"messages": state["messages"]}, {"tags": [f"attempt:{attempt}"]}
            )
            try:
                self.validator.invoke(response)
                return {"messages": response}
            except ValidationError as e:
                state = state + [
                    response,
                    ToolMessage(
                        content=f"{repr(e)}\n\nPay close attention to the function schema.\n\n"
                        + self.validator.schema_json()
                        + " Respond by fixing all validation errors.",
                        tool_call_id=response.tool_calls[0]["id"],
                    ),
                ]
        return {"messages": response}

In [ ]:
import datetime

actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are expert researcher.
Current time: {time}

1. {first_instruction}
2. Reflect and critique your answer. Be severe to maximize improvement.
3. Recommend search queries to research information and improve your answer.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "\n\n<system>Reflect on the user's original question and the"
            " actions taken thus far. Respond using the {function_name} function.</reminder>",
        ),
    ]
).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)
initial_answer_chain = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer.",
    function_name=AnswerQuestion.__name__,
) | llm.bind_tools(tools=[AnswerQuestion])
validator = PydanticToolsParser(tools=[AnswerQuestion])

first_responder = ResponderWithRetries(
    runnable=initial_answer_chain, validator=validator
)

In [ ]:
example_question = "Why is reflection useful in AI?"
initial = first_responder.respond(
    {"messages": [HumanMessage(content=example_question)]}
)

#### 수정 단계

액터의 두 번째 단계는 답변을 수정하는 과정입니다.

In [ ]:
revise_instructions = """새로운 정보를 활용해 이전 답변을 수정하세요.
    - 이전 비판을 활용하여 중요한 정보를 추가합니다.
        - 수정된 답변에는 검증이 가능하도록 반드시 번호가 매겨진 인용을 포함해야 합니다.
        - 답변 하단에 "References" 섹션을 추가합니다(단어 수에 포함되지 않음). 예시 형식:
            - [1] https://example.com
            - [2] https://example.com
    - 이전 비판을 참고하여 불필요한 정보를 제거하고, 답변이 250단어를 넘지 않도록 하세요.
"""


# Extend the initial answer schema to include references.
# Forcing citation in the model encourages grounded responses
class ReviseAnswer(AnswerQuestion):
    """Revise your original answer to your question. Provide an answer, reflection,

    cite your reflection with references, and finally
    add search queries to improve the answer."""

    references: list[str] = Field(
        description="Citations motivating your updated answer."
    )


revision_chain = actor_prompt_template.partial(
    first_instruction=revise_instructions,
    function_name=ReviseAnswer.__name__,
) | llm.bind_tools(tools=[ReviseAnswer])
revision_validator = PydanticToolsParser(tools=[ReviseAnswer])

revisor = ResponderWithRetries(runnable=revision_chain, validator=revision_validator)

In [ ]:
import json

revised = revisor.respond(
    {
        "messages": [
            HumanMessage(content=example_question),
            initial["messages"],
            ToolMessage(
                tool_call_id=initial["messages"].tool_calls[0]["id"],
                content=json.dumps(
                    tavily_tool.invoke(
                        {
                            "query": initial["messages"].tool_calls[0]["args"][
                                "search_queries"
                            ][0]
                        }
                    )
                ),
            ),
        ]
    }
)
revised["messages"]

## 도구 노드 만들기

다음으로 도구 호출을 실행할 노드를 만듭니다. LLM마다 서로 다른 스키마 이름을 사용하지만(검증에도 활용), 실제로는 동일한 도구로 연결되도록 합니다.

In [ ]:
from langchain_core.tools import StructuredTool

from langgraph.prebuilt import ToolNode


def run_queries(search_queries: list[str], **kwargs):
    """Run the generated queries."""
    return tavily_tool.batch([{"query": query} for query in search_queries])


tool_node = ToolNode(
    [
        StructuredTool.from_function(run_queries, name=AnswerQuestion.__name__),
        StructuredTool.from_function(run_queries, name=ReviseAnswer.__name__),
    ]
)

## 그래프 구성


이제 모든 구성 요소를 하나로 연결해 보겠습니다.

In [ ]:
from typing import Literal

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


MAX_ITERATIONS = 5
builder = StateGraph(State)
builder.add_node("draft", first_responder.respond)


builder.add_node("execute_tools", tool_node)
builder.add_node("revise", revisor.respond)
# draft -> execute_tools
builder.add_edge("draft", "execute_tools")
# execute_tools -> revise
builder.add_edge("execute_tools", "revise")

# Define looping logic:


def _get_num_iterations(state: list):
    i = 0
    for m in state[::-1]:
        if m.type not in {"tool", "ai"}:
            break
        i += 1
    return i


def event_loop(state: list):
    # in our case, we'll just stop after N plans
    num_iterations = _get_num_iterations(state["messages"])
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"


# revise -> execute_tools OR end
builder.add_conditional_edges("revise", event_loop, ["execute_tools", END])
builder.add_edge(START, "draft")
graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
events = graph.stream(
    {"messages": [("user", "How should we handle the climate crisis?")]},
    stream_mode="values",
)
for i, step in enumerate(events):
    print(f"Step {i}")
    step["messages"][-1].pretty_print()

## 마무리

리플렉션 액터를 구축하느라 수고하셨습니다! 워크플로에 어떤 부분을 적용할지 결정할 때 도움이 될 만한 몇 가지 관찰을 남겨드립니다.
1. 이 에이전트는 실행 시간과 품질을 맞바꾸는 구조입니다. 여러 단계를 거치며 스스로를 비판하고 수정하도록 강제하기 때문에 보통(항상은 아니지만) 응답 품질이 향상되지만 최종 답변이 나오기까지 시간이 더 오래 걸립니다.
2. 이러한 "반성" 단계는 검증기와 같은 추가 외부 피드백과 결합하여 액터를 더 효과적으로 안내할 수 있습니다.
3. 논문에서 한 환경(AlfWorld)은 외부 메모리를 사용합니다. 반성 내용을 요약해 외부 저장소에 보관한 뒤, 이후 시도나 호출에서 이를 활용합니다.